# 4. Optimizing and profiling the model

TODO: Add description

In [1]:
import timeit
import pandas as pd
import modelx as mx

In [2]:
ex1 = mx.read_model('CashValue_ME_EX1') 

In [3]:
timeit.timeit('ex1.Projection.result_pv()', globals=globals(), number=1)

1.2689610000234097

In [4]:
ex1.clear_all()

In [5]:
ex4 = mx.read_model('CashValue_ME_EX4') 

In [6]:
timeit.timeit('ex4.Projection.result_pv()', globals=globals(), number=1)

0.526920499978587

In [7]:
ex4.clear_all()

TODO: Add description

## Testing with 100,000 scenarios

TODO: Add description

<div class="alert alert-warning">

**Warning:**

takes about 4GB of memory space 
    
</div>

In [8]:
ex1.Projection.scen_size = 100_000

In [9]:
timeit.timeit('ex1.Projection.result_pv()', globals=globals(), number=1)

7.84547259984538

In [10]:
ex1.clear_all()

In [11]:
ex4.Projection.scen_size = 100_000

In [12]:
timeit.timeit('ex4.Projection.result_pv()', globals=globals(), number=1)

3.8828028000425547

In [13]:
ex4.clear_all()

## Profile the runs

TODO: Add description

In [14]:
def get_time_info(model):
    
    try:
        mx.start_stacktrace(maxlen=None)
        model.Projection.result_pv()
        df = pd.DataFrame.from_dict(
            mx.get_stacktrace(summarize=True), orient="index")
    finally:
        mx.stop_stacktrace()
        model.clear_all()

    # Remove model and space names from index
    prefixlen = len(model.name + '.Projection.')
    df.index = [s[prefixlen:] for s in df.index]     

    # Add duration %
    total = df['duration'].sum()
    df['dur_perc'] = df['duration'] * 100 / total
    
    return df.sort_values(['dur_perc'], ascending=False)

In [15]:
ex1.Projection.scen_size = 10_000
df1 = get_time_info(ex1)
df1[['duration', 'dur_perc']].iloc[:10]

,duration,dur_perc
surr_charge_rate(t),0.626154,36.491786
inv_return_mth(t),0.291798,17.005771
premium_pp(t),0.206414,12.029663
"claim_pp(t, kind)",0.077510,4.517216
"claims(t, kind)",0.063502,3.700866
"pols_if_at(t, timing)",0.038705,2.255718
"av_pp_at(t, timing)",0.034805,2.028384
inv_return_table(),0.031250,1.821226
expenses(t),0.029133,1.697868
inv_income(t),0.027631,1.610330


In [16]:
ex4.Projection.scen_size = 10_000
df4 = get_time_info(ex4)
df4[['duration', 'dur_perc']].iloc[:10]

,duration,dur_perc
surr_charge_rate(t),0.204015,37.341396
premium_pp(t),0.115471,21.134984
pv_claims(kind),0.031336,5.735535
inv_return_table(),0.023608,4.320994
lapse_rate(t),0.022686,4.152201
prem_to_av_pp(t),0.018686,3.420122
av_change(t),0.016086,2.944244
commissions(t),0.015623,2.859585
premiums(t),0.015507,2.838290
std_norm_rand(),0.012620,2.309915


In [17]:
df = pd.concat([df1[['duration']], df4[['duration']]], axis=1)
df.columns = ['df1', 'df4']
df.loc[(df['df1'] > 0.05) | (df['df4'] > 0.05)]

,df1,df4
surr_charge_rate(t),0.626154,0.204015
inv_return_mth(t),0.291798,0.002037
premium_pp(t),0.206414,0.115471
"claim_pp(t, kind)",0.077510,0.002008
"claims(t, kind)",0.063502,0.000000
